In [1]:
# Libs
import os
import pandas as pd
from download_transformacao_CNPJ import EXTRATOR_CNPJ
from pyspark.sql.functions import concat_ws, lpad, coalesce, when, lit
from time import localtime, strftime
current_dir = os.getcwd()
dir_dados = os.path.join(current_dir, r"dados")

In [2]:
# Se passar baixar_e_extrair como false, precisa do nome do arquivo.
print(f'Começando a buscar os dados: {strftime("%d/%m/%Y %H:%M:%S", localtime())}')
ESTABELECIMENTOS, spark = EXTRATOR_CNPJ(baixar_e_extrair=True, nome_arquivo="Estabelecimentos").run()
print(f'Termino da coleta dos ESTABELECIMENTOS: {strftime("%d/%m/%Y %H:%M:%S", localtime())}')
EMPRESAS, spark = EXTRATOR_CNPJ(baixar_e_extrair=True, nome_arquivo="Empresas").run()
print(f'Termino da coleta dos EMPRESAS: {strftime("%d/%m/%Y %H:%M:%S", localtime())}')
MUNICIPIOS, spark = EXTRATOR_CNPJ(baixar_e_extrair=True, nome_arquivo="Municipios").run()
print(f'Termino da coleta dos MUNICIPIOS: {strftime("%d/%m/%Y %H:%M:%S", localtime())}')
SOCIOS, spark = EXTRATOR_CNPJ(baixar_e_extrair=True, nome_arquivo="Socios").run()
print(f'Final da coleta dos dados: {strftime("%d/%m/%Y %H:%M:%S", localtime())}')

Começando a buscar os dados: 14/07/2023 14:36:42
Termino da coleta dos ESTABELECIMENTOS: 14/07/2023 16:00:05
Termino da coleta dos EMPRESAS: 14/07/2023 16:11:02
Termino da coleta dos MUNICIPIOS: 14/07/2023 16:11:03
Final da coleta dos dados: 14/07/2023 16:15:28


In [7]:
SIMPLES, spark = EXTRATOR_CNPJ(baixar_e_extrair=True, nome_arquivo="Simples").run()

In [15]:
SIMPLES.describe()

DataFrame[summary: string, CNPJ_BASE: string, OPCAO_SIMPLES: string, DATA_OPCAO_SIMPLES: string, DATA_EXCLUSAO_SIMPLES: string, OPCAO_MEI: string, DATA_OPCAO_MEI: string, DATA_EXCLUSAO_MEI: string]

In [4]:
ESTABELECIMENTOS.show(1, vertical=True)

-RECORD 0------------------------------------------------
 CNPJ_BASE                        | 23615079             
 CNPJ_ORDEM                       | 1                    
 CNPJ_DV                          | 60                   
 MATRIZ_FILIAL                    | 1                    
 NOME_FANTASIA                    | KING FOODS DISTRI... 
 SITUACAO_CADASTRAL               | 08                   
 DATA_SITUACAO_CADASTRAL          | 20190115             
 CODIGO_MOTIVO_SITUACAO_CADASTRAL | 1                    
 CIDADE_EXTERIOR                  | null                 
 CODIGO_PAIS                      | null                 
 DATA_INICIO_ATIVIDADE            | 20151106             
 CNAE_PRINCIPAL                   | 4691500              
 CNAE_SECUNDARIO                  | null                 
 TIPO_LOGRADOURO                  | RUA                  
 LOGRADOURO                       | EXPEDICIONARIO JO... 
 NUMERO                           | 512                  
 COMPLEMENTO  

In [5]:
EMPRESAS.show(1, vertical=True)

-RECORD 0----------------------------------------
 CNPJ_BASE                | 41273597             
 RAZAO_SOCIAL             | PACHARRUS QUEIROZ... 
 CODIGO_NATUREZA_JURIDICA | 2135                 
 CODIGO_QUALIFICACAO      | 50                   
 CAPITAL_SOCIAL           | 5000,00              
 PORTE_EMPRESA            | 1                    
 ENTE_FEDERATIVO          | null                 
 NOME_ARQUIVO             | rfb/RAW/EMPRESAS/... 
only showing top 1 row



In [6]:
SOCIOS.show(1, vertical=True)

-RECORD 0-------------------------------------------------
 CNPJ_BASE                         | 46204351             
 TIPO_SOCIO                        | 2                    
 NOME_SOCIO                        | RUTHIELLI DOS SAN... 
 CPF_CNPJ                          | ***627370**          
 CODIGO_QUALIFICACAO_SOCIO         | 49                   
 DATA_ENTRADA_SOCIEDADE            | 20220429             
 PERCENTUAL_CAPITAL_SOCIAL         | null                 
 CPF_REPRESENTANTE                 | ***000000**          
 NOME_REPRESENTANTE                | null                 
 CODIGO_QUALIFICACAO_REPRESENTANTE | 0                    
 FAIXA_ETARIA                      | 3                    
 NOME_ARQUIVO                      | rfb/RAW/SOCIOS/CN... 
only showing top 1 row



In [3]:
CNAES = {
        5620104:'Fornecimento de alimentos preparados preponderantemente para consumo domiciliar',
        5611201:'Restaurantes e similares',
        5611203:'Lanchonetes casas de chá de sucos e similares',
        5611204:'Bares e outros estabelecimentos especializados em servir bebidas sem entretenimento',
        5611205:'Bares e outros estabelecimentos especializados em servir bebidas com entretenimento',
        4721102:'Padaria e confeitaria com predominância de revenda'
        }

In [ ]:
cnaes = pd.DataFrame(CNAES.items(),columns=["CNAE_PRINCIPAL", "CNAE_DESCRICAO"], dtype="string")
cnaes

# SQL QUERY SPARK
``` PYTHON
# Cria uma view com o mesmo nome do DataFrame
ESTABELECIMENTOS.createOrReplaceTempView("ESTABELECIMENTOS")
# Contagem ativos, inativos e geral
spark.sql("""SELECT t1.CNAE_PRINCIPAL, t1.QUANTIDADE_FECHADA, t2.QUANTIDADE_GERAL, t3.QUANTIDADE_ABERTA
            FROM (
            SELECT CNAE_PRINCIPAL, COUNT(*) AS QUANTIDADE_FECHADA
            FROM ESTABELECIMENTOS
            WHERE CNAE_PRINCIPAL IN ('4721102','5612100', '5611201', '5611203', '5611204', '5611205') AND SITUACAO_CADASTRAL NOT IN (2, 3, 4)
            GROUP BY CNAE_PRINCIPAL
            ) AS t1
            FULL OUTER JOIN (
            SELECT CNAE_PRINCIPAL, COUNT(*) AS QUANTIDADE_GERAL
            FROM ESTABELECIMENTOS 
            WHERE CNAE_PRINCIPAL IN ('4721102','5612100', '5611201', '5611203', '5611204', '5611205')
            GROUP BY CNAE_PRINCIPAL 
            ) AS t2 
            ON t1.CNAE_PRINCIPAL = t2.CNAE_PRINCIPAL
            FULL OUTER JOIN (
            SELECT CNAE_PRINCIPAL, COUNT(*) AS QUANTIDADE_ABERTA
            FROM ESTABELECIMENTOS
            WHERE CNAE_PRINCIPAL IN ('4721102','5612100', '5611201', '5611203', '5611204', '5611205') AND SITUACAO_CADASTRAL IN (2, 3, 4)
            GROUP BY CNAE_PRINCIPAL
            ) AS t3 
            ON t1.CNAE_PRINCIPAL = t3.CNAE_PRINCIPAL
            ORDER BY t1.CNAE_PRINCIPAL
            LIMIT 100
            """).show()
```

In [20]:
def filtragem_cnae_sql(cod_cnae: int):
    # Cria uma view com o mesmo nome do DataFrame
    ESTABELECIMENTOS.createOrReplaceTempView("ESTABELECIMENTOS")
    EMPRESAS.createOrReplaceTempView("EMPRESAS")
    MUNICIPIOS.createOrReplaceTempView("MUNICIPIOS")
    SIMPLES.createOrReplaceTempView("SIMPLES")
    SOCIOS.createOrReplaceTempView("SOCIOS")
    # cria um dataframe com base na query
    dataframe = spark.sql(
        f"""
        SELECT CONCAT(LPAD(EST.CNPJ_BASE, 8, '0'), LPAD(EST.CNPJ_ORDEM, 4, '0'), LPAD(EST.CNPJ_DV, 2, '0')) AS CNPJ,
                EMP.RAZAO_SOCIAL,
                EST.NOME_FANTASIA,
                EST.SITUACAO_CADASTRAL,
                EST.DATA_SITUACAO_CADASTRAL,
                EST.DATA_INICIO_ATIVIDADE,
                SOC.NOME_SOCIO,
                SIM.OPCAO_MEI,
                SIM.DATA_OPCAO_MEI,
                SIM.DATA_EXCLUSAO_MEI,
                EST.CNAE_PRINCIPAL,
                CONCAT(
                    COALESCE(EST.TIPO_LOGRADOURO, ''),
                    ' ',
                    COALESCE(EST.LOGRADOURO, ''),
                    ' ',
                    COALESCE(EST.NUMERO, ''),
                    ' ',
                    COALESCE(EST.COMPLEMENTO, '')
                ) AS ENDERECO
                EST.BAIRRO,
                MUN.NOME_MUNICIPIO AS CIDADE,
                EST.UF,
                EST.CEP,
                CONCAT(
                    COALESCE(EST.DDD_CONTATO, ''), 
                    ' ',
                    COALESCE(EST.TELEFONE_CONTATO, '')
                    ) AS TELEFONE,
                EST.EMAIL
        FROM ESTABELECIMENTOS AS EST, EMPRESAS AS EMP, MUNICIPIOS AS MUN, SIMPLES AS SIM, SOCIOS AS SOC
        WHERE EST.CNPJ_BASE = EMP.CNPJ_BASE
            AND EST.CNPJ_BASE = SIM.CNPJ_BASE
            AND EST.CNPJ_BASE = SOC.CNPJ_BASE
            AND EST.CODIGO_MUNICIPIO = MUN.CODIGO_MUNICIPIO
            AND EST.CNAE_PRINCIPAL = {cod_cnae}
            AND EST.SITUACAO_CADASTRAL IN (2, 3, 4)
        """
    )
    #display(dataframe.show(5))
    return dataframe

In [27]:
def filtragem_cnae_df(cod_cnae:int):
    # cria um dataframe com base nos filtros aplicados
    from pyspark.sql.functions import concat_ws, lpad, coalesce, when
    dataframe = (
        ESTABELECIMENTOS
        .join(EMPRESAS, "CNPJ_BASE", "right")
        .join(MUNICIPIOS, "CODIGO_MUNICIPIO", "right")
        .join(SIMPLES, "CNPJ_BASE", "right")
        .join(SOCIOS, "CNPJ_BASE", "right")
        .where(
            (ESTABELECIMENTOS["CNAE_PRINCIPAL"] == f"{cod_cnae}") &
            (ESTABELECIMENTOS["SITUACAO_CADASTRAL"].isin([2, 3, 4]))|
            (ESTABELECIMENTOS["CNAE_SECUNDARIO"].isin(cod_cnae)) &
            (ESTABELECIMENTOS["SITUACAO_CADASTRAL"].isin([2, 3, 4]))
        )
        .select(
            concat_ws("", 
                lpad(ESTABELECIMENTOS["CNPJ_BASE"].cast("bigint"), 8, "0"), 
                lpad(ESTABELECIMENTOS["CNPJ_ORDEM"], 4, "0"), 
                lpad(ESTABELECIMENTOS["CNPJ_DV"], 2, "0")
            ).alias("CNPJ"),
            when(EMPRESAS.RAZAO_SOCIAL.isNull(), None).otherwise(EMPRESAS.RAZAO_SOCIAL).alias("RAZAO_SOCIAL"),
            when(ESTABELECIMENTOS.NOME_FANTASIA.isNull(), None).otherwise(ESTABELECIMENTOS.NOME_FANTASIA).alias("NOME_FANTASIA"),
            ESTABELECIMENTOS.SITUACAO_CADASTRAL,
            ESTABELECIMENTOS.DATA_SITUACAO_CADASTRAL,
            ESTABELECIMENTOS.DATA_INICIO_ATIVIDADE,
            when(SIMPLES.OPCAO_MEI.isNull(), None).otherwise(SIMPLES.OPCAO_MEI).alias("OPCAO_MEI"),
            when(SOCIOS.NOME_SOCIO.isNull(), None).otherwise(SOCIOS.NOME_SOCIO).alias("NOME_SOCIO"),
            when(SIMPLES.DATA_EXCLUSAO_MEI.isNull(), None).otherwise(SIMPLES.DATA_EXCLUSAO_MEI).alias("DATA_EXCLUSAO_MEI"),
            when(SIMPLES.DATA_OPCAO_MEI.isNull(), None).otherwise(SIMPLES.DATA_OPCAO_MEI).alias("DATA_OPCAO_MEI"),
            ESTABELECIMENTOS.CNAE_PRINCIPAL,
            ESTABELECIMENTOS.CNAE_SECUNDARIO,
            concat_ws(" ",
                when(ESTABELECIMENTOS.TIPO_LOGRADOURO.isNull(), None).otherwise(ESTABELECIMENTOS.TIPO_LOGRADOURO),
                when(ESTABELECIMENTOS.LOGRADOURO.isNull(), None).otherwise(ESTABELECIMENTOS.LOGRADOURO),
                when(ESTABELECIMENTOS.NUMERO.isNull(), None).otherwise(ESTABELECIMENTOS.NUMERO),
                when(ESTABELECIMENTOS.COMPLEMENTO.isNull(), None).otherwise(ESTABELECIMENTOS.COMPLEMENTO)
            ).alias("ENDERECO"),
            ESTABELECIMENTOS.BAIRRO,
            MUNICIPIOS.NOME_MUNICIPIO.alias("CIDADE"),
            ESTABELECIMENTOS.UF,
            ESTABELECIMENTOS.CEP,
            concat_ws("-", 
                when(ESTABELECIMENTOS.DDD_CONTATO.isNull(), None).otherwise(ESTABELECIMENTOS.DDD_CONTATO),
                when(ESTABELECIMENTOS.TELEFONE_CONTATO.isNull(), None).otherwise(ESTABELECIMENTOS.TELEFONE_CONTATO)
            ).alias("TELEFONE"),
            ESTABELECIMENTOS.EMAIL
        )
    )

    #display(dataframe.show(5))
    return dataframe

In [22]:
def filtra_cnae_lista(lista_cnaes:list):
    # cria um dataframe com base nos filtros aplicados
    from pyspark.sql.functions import concat_ws, lpad, coalesce, when
    dataframe = (
        ESTABELECIMENTOS
        .join(EMPRESAS, "CNPJ_BASE", "right")
        .join(MUNICIPIOS, "CODIGO_MUNICIPIO", "right")
        .where(
            (ESTABELECIMENTOS["CNAE_PRINCIPAL"].isin(lista_cnaes)) &
            (ESTABELECIMENTOS["SITUACAO_CADASTRAL"].isin([2, 3, 4])) |
            (ESTABELECIMENTOS["CNAE_SECUNDARIO"].isin(lista_cnaes)) &
            (ESTABELECIMENTOS["SITUACAO_CADASTRAL"].isin([2, 3, 4]))
        )
        .select(
            concat_ws("", 
                lpad(ESTABELECIMENTOS["CNPJ_BASE"].cast("bigint"), 8, "0"), 
                lpad(ESTABELECIMENTOS["CNPJ_ORDEM"], 4, "0"), 
                lpad(ESTABELECIMENTOS["CNPJ_DV"], 2, "0")
            ).alias("CNPJ"),
            when(EMPRESAS.RAZAO_SOCIAL.isNull(), None).otherwise(EMPRESAS.RAZAO_SOCIAL).alias("RAZAO_SOCIAL"),
            when(ESTABELECIMENTOS.NOME_FANTASIA.isNull(), None).otherwise(ESTABELECIMENTOS.NOME_FANTASIA).alias("NOME_FANTASIA"),
            ESTABELECIMENTOS.SITUACAO_CADASTRAL,
            ESTABELECIMENTOS.DATA_SITUACAO_CADASTRAL,
            ESTABELECIMENTOS.DATA_INICIO_ATIVIDADE,
            when(SIMPLES.OPCAO_MEI.isNull(), None).otherwise(SIMPLES.OPCAO_MEI).alias("OPCAO_MEI"),
            when(SOCIOS.NOME_SOCIO.isNull(), None).otherwise(SOCIOS.NOME_SOCIO).alias("NOME_SOCIO"),
            when(SIMPLES.DATA_EXCLUSAO_MEI.isNull(), None).otherwise(SIMPLES.DATA_EXCLUSAO_MEI).alias("DATA_EXCLUSAO_MEI"),
            when(SIMPLES.DATA_OPCAO_MEI.isNull(), None).otherwise(SIMPLES.DATA_OPCAO_MEI).alias("DATA_OPCAO_MEI"),
            ESTABELECIMENTOS.CNAE_PRINCIPAL,
            ESTABELECIMENTOS.CNAE_SECUNDARIO,
            concat_ws(" ",
                when(ESTABELECIMENTOS.TIPO_LOGRADOURO.isNull(), None).otherwise(ESTABELECIMENTOS.TIPO_LOGRADOURO),
                when(ESTABELECIMENTOS.LOGRADOURO.isNull(), None).otherwise(ESTABELECIMENTOS.LOGRADOURO),
                when(ESTABELECIMENTOS.NUMERO.isNull(), None).otherwise(ESTABELECIMENTOS.NUMERO),
                when(ESTABELECIMENTOS.COMPLEMENTO.isNull(), None).otherwise(ESTABELECIMENTOS.COMPLEMENTO)
            ).alias("ENDERECO"),
            ESTABELECIMENTOS.BAIRRO,
            MUNICIPIOS.NOME_MUNICIPIO.alias("CIDADE"),
            ESTABELECIMENTOS.UF,
            ESTABELECIMENTOS.CEP,
            concat_ws("-", 
                when(ESTABELECIMENTOS.DDD_CONTATO.isNull(), None).otherwise(ESTABELECIMENTOS.DDD_CONTATO),
                when(ESTABELECIMENTOS.TELEFONE_CONTATO.isNull(), None).otherwise(ESTABELECIMENTOS.TELEFONE_CONTATO)
            ).alias("TELEFONE"),
            ESTABELECIMENTOS.EMAIL
        )
    )

    #display(dataframe.show(5))
    return dataframe

### Salva no disco e lê após isso
``` PYTHON
# Libs
import requests # SE FOR USAR REQUESTS
import urllib.request # SE FOR USAR URLLIB
from pySmartDL import SmartDL # SE FOR USAR PYSMARTDL
import zipfile
import os
from pyspark.sql import SparkSession
current_dir = os.getcwd()

# Define ou busca uma sessão do Spark
spark = SparkSession.builder.master("local[2]") \
    .appName("OnlineReader") \
    .getOrCreate()

# Define a url de download dos dados
url = 'https://dadosabertos.rfb.gov.br/CNPJ/Simples.zip'
# Pega o nome do arquivo pela url
arquivo = url.split('/')[-1]
# Define o caminho absoluto do diretório.
salvar_onde = f"{current_dir}/RAW/{arquivo.split('.')[0]}/"

# cria a pasta para armazenar o arquivo, se ela não existir
if not os.path.exists(salvar_onde):
    os.makedirs(salvar_onde)

### Com requests
"""
    with requests.get(url, stream=True) as response:
        with open(os.path.join(salvar_onde, arquivo), 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

"""
### Com urllib

# faz o download do arquivo e salva em salvar_onde/arquivo
urllib.request.urlretrieve(url, os.path.join(salvar_onde, arquivo))

### Com SmartDL
"""
    dest = os.path.join(salvar_onde, arquivo)
    obj = SmartDL(url, dest, threads=4)
    obj.start()
"""

"""
# Imprime o caminho do diretório de download
print(salvar_onde)
"""

# Descompactação do arquivo
with zipfile.ZipFile(os.path.join(salvar_onde, arquivo), 'r') as zip_ref:

    # obtem o nome do primeiro arquivo dentro do zip
    nome_original_arquivo_zip = zip_ref.namelist()[0]

    # define um novo nome para o arquivo
    novo_nome_arquivo = f"CNPJ_{arquivo.split('.')[0]}.csv"

    # cria um dicionário com as informações de origem e destino
    arquivos_para_extrair = {nome_original_arquivo_zip : novo_nome_arquivo}

    # realiza a extração do arquivo zip
    zip_ref.extractall(path = f"{salvar_onde}/", members=arquivos_para_extrair)
    
    # renomeia o arquivo extraído com o novo nome
    os.rename(os.path.join(salvar_onde, nome_original_arquivo_zip), os.path.join(salvar_onde, novo_nome_arquivo))

"""
# imprime o nome do arquivo dentro do zip e o novo nome
print(f"Arquivo dentro do zip: {nome_original_arquivo_zip}")
print(f"Novo nome do arquivo: {novo_nome_arquivo}")
"""

# Define o caminho absoluto para o arquivo
csv_file = os.path.join(salvar_onde, novo_nome_arquivo)

# Lê o arquivo em um dataframe Spark
dados = spark.read.options(delimiter=";", header=False, inferSchema=True).csv(csv_file)

# Plota primeira linha do dataframe
dados.show(1, vertical=True)
```

In [28]:
def salvar_df_cnae(CNAES:dict[int,str] = CNAES, opcao:int = None):
    from backup_limpeza import backup_limpeza_simples
    """
    Args:
        CNAES (dict[int,str], optional): informa um dicionário com os códigos e descrição cnae.
    Return:
        dados_pandas : salva o dataframe gerador pela função em um arquivo único csv e arquivos parquets
    """
    arquivo_csv = os.path.join(dir_dados, r"csv\BASE_RFB_MEI.csv")
    if os.path.exists(arquivo_csv):
        nome_backup = dir_dados + r"/backup/"
        if not os.path.exists(nome_backup):
            os.makedirs(nome_backup)
        backup_limpeza_simples(pasta=arquivo_csv.replace(r"BASE_RFB_MEI.csv", ""), nome_zipado=nome_backup + f"BASE_RFB_{strftime('%d-%m-%Y %H_%M_%S', localtime())}.zip")
    if opcao == 1:
        dados = None
        for cod_cnae, descricao_cnae in CNAES.items():
            if dados is None:
                print(f"{cod_cnae}:{descricao_cnae}")
                dados = filtragem_cnae_df(cod_cnae)
                dados = dados.withColumn(
                                "CNAE_DESCRICAO", lit(descricao_cnae.upper())
                            )
            else:
                print(f"{cod_cnae}:{descricao_cnae}")
                dados_incrementados = filtragem_cnae_df(cod_cnae)
                dados_incrementados = dados_incrementados.withColumn(
                    "CNAE_DESCRICAO", lit(descricao_cnae.upper())
                )
                dados = dados.union(dados_incrementados)
                dados = dados.sort("CNPJ")
        dados_pandas = dados.toPandas()
        print(len(dados_pandas.index))
        dados_pandas.to_csv(arquivo_csv, sep=";",mode="a", encoding="utf-8", index=False)
    elif opcao == 2:
        lista_cnaes = list(CNAES.keys())
        dados = filtra_cnae_lista(lista_cnaes)
        cnaes = pd.DataFrame(CNAES.items(),columns=["CNAE_PRINCIPAL", "CNAE_DESCRICAO"], dtype="string")
        dados_pandas = dados.toPandas()
        dados_pandas = pd.merge(dados_pandas, cnaes, on="CNAE_PRINCIPAL")
        
        dados_pandas = dados_pandas[["CNPJ","RAZAO_SOCIAL","NOME_FANTASIA",
            "SITUACAO_CADASTRAL","DATA_SITUACAO_CADASTRAL",
            "DATA_INICIO_ATIVIDADE","NOME_SOCIO","OPCAO_MEI","DATA_EXCLUSAO_MEI","DATA_OPCAO_MEI","CNAE_PRINCIPAL","ENDERECO",
            "BAIRRO","CIDADE","UF","CEP","TELEFONE","CNAE_DESCRICAO", "EMAIL"]]
        print(len(dados_pandas.index))
        dados_pandas.to_csv(arquivo_csv, sep=";",mode="a", encoding="utf-8", index=False)
    else:
        pass

In [30]:
salvar_df_cnae(CNAES, opcao=1)

5620104:Fornecimento de alimentos preparados preponderantemente para consumo domiciliar
5611201:Restaurantes e similares
5611203:Lanchonetes casas de chá de sucos e similares
5611204:Bares e outros estabelecimentos especializados em servir bebidas sem entretenimento
5611205:Bares e outros estabelecimentos especializados em servir bebidas com entretenimento
4721102:Padaria e confeitaria com predominância de revenda


# Define função que manipula a list ade arquivos
```PYTHON
def manipula_lista_arquivos(lista_de_arquivos:list):
    # Lê cada arquivo parquet em um dataframe Spark 
    dados = None
    for arquivo_no_diretorio in lista_de_arquivos:
        if dados is None:
            dados = (
                spark.read.format("parquet")
                .option("inferSchema", "true")
                .load(arquivo_no_diretorio)
            )
        else:
            # print(arquivo_no_diretorio)
            dados_incrementados = (
                spark.read.format("parquet")
                .option("inferSchema", "true")
                .load(arquivo_no_diretorio)
            )
            dados = dados.union(dados_incrementados)
    
    pd_dados = dados.toPandas()
    #pd_dados = dados.toPandas()
    return pd_dados```

``` PYTHON
dir_parquet = dir_dados + r"\parquet"
lista_arquivos_no_diretorio = [
            os.path.join(dir_parquet, nome)
            for nome in os.listdir(dir_parquet)
            if nome.endswith(".parquet")
        ]
df_to_save = manipula_lista_arquivos(lista_arquivos_no_diretorio)```

```PYTHON
arquivo_csv = os.path.join(dir_dados, r"csv\BASE_RFB.csv")
df_to_save.to_csv(arquivo_csv, sep=";", encoding="utf-8", index=False)```